#  Recommendation Systems

Load Pandas, we are going to need it for manipulating data

In [276]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from typing import Optional, List, Tuple, Dict, Any, NoReturn, Iterable, Union, Literal

import matplotlib.pyplot as plt

%matplotlib inline

np.set_printoptions(precision=3)

_rng: np.random.Generator = np.random.default_rng(42)

In [224]:
colnames = ["rated"] + [i for i in range(1,101)]



data: pd.DataFrame = pd.read_csv(
    "jester-data-1.csv",
    #names=colnames,
    header=None
)
#data.drop("rated", axis=1, inplace=True)

data.drop(0, inplace=True)
data.replace(99, np.NaN, inplace=True)


data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
1,100.0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49.0,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,48.0,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,91.0,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
5,100.0,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [225]:
print(data.shape)


(24982, 101)


In [226]:
means: pd.Series = data.mean()

print(means)

print(f"Highest rated: {means.idxmax()} {means[means.idxmax()]}")
print(f"Lowest  rated: {means.idxmin()} {means[means.idxmin()]}")


0      72.381109
1       0.905104
2       0.207988
3       0.317174
4      -1.448684
         ...    
96      1.528003
97      1.674444
98      0.767692
99     -0.031244
100     1.354711
Length: 101, dtype: float64
Highest rated: 0 72.38110919240191
Lowest  rated: 58 -3.833520447530859


In [227]:
original_data: pd.DataFrame = data.copy()

""

''

In [309]:
def make_validation_set(
        original: pd.DataFrame, replace_proportion: float = 0.1, replace_with: Any = 99
) -> Tuple[pd.DataFrame, Tuple[np.ndarray], Tuple[np.ndarray]]:
    """
    Creates a validation set version of the dataset.
    Obtains replace_proportion of the non NaN values in the dataset,
    and with those specific selected non-NaNs, replaces
    them with replace_with in that dataset.
    :param original: 
    :param size_percent: 
    :param replace_with:
    :return: 
    """
    valid_set: pd.DataFrame = original.copy()
    rated = np.where(~np.isnan(valid_set.values))
    n_rated: int = len(rated[0])
    replace_these = _rng.choice(n_rated, size=int(replace_proportion*n_rated), replace=False)

    print(replace_these)

    print(rated)



    valid_vals: np.ndarry = valid_set.values.flatten()
    valid_vals.flatten()[replace_these] = replace_with


    
    nintetynines = np.where(valid_vals==replace_with)

    print(nintetynines)

    #for x, y in zip(replace_these[0][i], ):
    #    valid_set[replace_these[i]]
    #valid_set.loc[rated[0][replace_these]],[rated[1][replace_these]] = replace_with
    return valid_set, (rated[0][replace_these], rated[1][replace_these])
    

v_orig: Tuple[pd.DataFrame, Tuple[np.ndarray], Tuple[np.ndarray]] = make_validation_set(
    data, 0.1, 99
)



validation: pd.DataFrame = v_orig[0]

count_99 = 0

for c in validation.columns:
    try:
        count_99 += validation[c].value_counts()[99]
    except Exception:
        pass

print(f"99s: {count_99}")


#validation


def sgd(iterations, validation_val = 99, train_val = np.NaN):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                #if not np.isnan(rating):
                if (not np.isnan(rating)) and (rating != train_val and rating != validation_val):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print('Iteration %d/%d:    MSE=%.3f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history


[1514684  203125  960596 ... 1522583  370175 1644993]
(array([    0,     0,     0, ..., 24981, 24981, 24981]), array([ 0,  1,  2, ..., 70, 71, 87]))
(array([], dtype=int64),)
99s: 0


In [301]:
#df_jester.iloc[user_x, joke_y].

print(data.iloc[19928, 32])


6.02


In [230]:

movies_df: pd.DataFrame = pd.read_excel("movies_latent_factors.xlsx")

movies_df

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [231]:
movies_users: pd.DataFrame = pd.read_excel("movies_latent_factors.xlsx",sheet_name="Users")

movies_users


,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [310]:
#In the movies dataset, what's the loading (i.e., value) of
# Factor14 for the movie with ID=24?

movies_df.loc[movies_df['Movie ID'] == 36657, ["Factor14"]]

,Factor14
97,-1.355542


In [312]:
#In the movies dataset, which of the following titles
# corresponds to the movie with the lowest loading of factor 12?

movies_df.loc[movies_df['Factor15'] == np.max(movies_df['Factor15']), :]


,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
51,604,The Matrix Reloaded (2003),0.503414,-2.00445,0.001084,-2.255945,0.882547,-0.1886,-0.235434,0.570028,-3.492897,-0.312214,0.902742,-2.063318,-1.895101,0.748458,2.120907


In [234]:

n_latent_factors = 15

#user_ratings = movies_users.values
# Initialise as random values
latent_user_preferences: pd.DataFrame = movies_users.copy()
latent_item_features: pd.DataFrame = movies_df.copy()

latent_user_preferences.set_index("User", inplace=True)
latent_item_features.drop("Title", axis=1,inplace=True)
latent_item_features.set_index("Movie ID", inplace=True)
""


''

In [235]:
latent_item_features

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
Movie ID,,,,,,,,,,,,,,,
11,-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
12,-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
13,-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
14,-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
22,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9806,0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
10020,1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
36657,0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790


In [236]:
latent_user_preferences


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
User,,,,,,,,,,,,,,,
4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983


In [237]:

latent_item_features.loc[11]

latent_user_preferences.loc[3806]

Factor1    -0.042344
Factor2    -0.040755
Factor3    -0.055720
Factor4    -0.073726
Factor5     0.018203
Factor6     0.091940
Factor7     0.123959
Factor8    -0.157403
Factor9    -0.020388
Factor10    0.082493
Factor11    0.223057
Factor12   -0.040915
Factor13   -0.130308
Factor14   -0.021816
Factor15    0.034549
Name: 3806, dtype: float64

In [316]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences.loc[user_id]
    item_preference = latent_item_features.loc[item_id]
    return user_preference.dot(item_preference)

predict_rating(4768,786)


-0.07799268400487341

In [317]:
predict_rating(4768,14)

0.08617567851569355

In [281]:
def get_ratings(
        user_id,
        show_top: Optional[int] = None,
        descending: bool = True
):
    """
    Attempts to work out what ratings a user would give a movie
    :param user_id: the user
    :param show_top: give this value to only show the top [whatever] movies
    :param descending: sort in descending order (top rated first)
    """

    predict_ratings = dict(
        (i, predict_rating(user_id, i))
        for i in latent_item_features.T.columns.values
    )

    sorted_ratings = sorted(
        [*predict_ratings.items()],
        key=lambda kv: kv[1],
        reverse=descending
    )

    if show_top is None:
        show_top = len(sorted_ratings)

    for kv in sorted_ratings[:show_top]:
        print(f"{kv[0]}: {movies_df.loc[movies_df['Movie ID'] == kv[0], ['Title'][0]].values}")
        print(f"\t{kv[1]}")

    return sorted_ratings

    #for i in latent_item_features.T.columns.values:
    #    print(f"{i}: {movies_df.loc[movies_df['Movie ID'] == i, ['Title']]}")
    #    print(f"\t{predict_rating(user_id, i)}")

In [321]:
get_ratings(4529)


""

597: ['Titanic (1997)']
	1.0404564360144615
238: ['The Godfather (1972)']
	0.674625860954049
278: ['The Shawshank Redemption (1994)']
	0.6486912913595948
155: ['The Dark Knight (2008)']
	0.6033013587670666
424: ["Schindler's List (1993)"]
	0.5218542511897091
13: ['Forrest Gump (1994)']
	0.4832062579101699
603: ['The Matrix (1999)']
	0.4780317054931237
274: ['The Silence of the Lambs (1991)']
	0.4545784969134926
272: ['Batman Begins (2005)']
	0.44718028305733837
680: ['Pulp Fiction (1994)']
	0.42149755921629606
557: ['Spider-Man (2002)']
	0.4189097741243446
857: ['Saving Private Ryan (1998)']
	0.4010316072745454
453: ['A Beautiful Mind (2001)']
	0.39291053678619675
558: ['Spider-Man 2 (2004)']
	0.3704292728612526
280: ['Terminator 2: Judgment Day (1991)']
	0.3438621395970047
329: ['Jurassic Park (1993)']
	0.33784818715566267
11: ['Star Wars: Episode IV - A New Hope (1977)']
	0.32232658528473557
1891: ['Star Wars: Episode V - The Empire Strikes Back (1980)']
	0.31577146202964645
98: ['Gl

''

In [328]:
titles_5429 = [
    k for k, v in get_ratings(4529)
    if movies_df.loc[movies_df['Movie ID'] == k, ['Title'][0]].values[0] in [
        "Finding Nemo (2003)",
        "Titanic (1997)",
        "Traffic (2000)",
        "The Bourne Supremacy (2004)",
        "Independence Day (a.k.a. ID4) (1996)",
        "Amelie (2001)",
        "The Dark Knight (2008)"
    ]
]


for t in titles_5429:
    print(movies_df.loc[movies_df['Movie ID'] == t, ['Title'][0]].values)

597: ['Titanic (1997)']
	1.0404564360144615
238: ['The Godfather (1972)']
	0.674625860954049
278: ['The Shawshank Redemption (1994)']
	0.6486912913595948
155: ['The Dark Knight (2008)']
	0.6033013587670666
424: ["Schindler's List (1993)"]
	0.5218542511897091
13: ['Forrest Gump (1994)']
	0.4832062579101699
603: ['The Matrix (1999)']
	0.4780317054931237
274: ['The Silence of the Lambs (1991)']
	0.4545784969134926
272: ['Batman Begins (2005)']
	0.44718028305733837
680: ['Pulp Fiction (1994)']
	0.42149755921629606
557: ['Spider-Man (2002)']
	0.4189097741243446
857: ['Saving Private Ryan (1998)']
	0.4010316072745454
453: ['A Beautiful Mind (2001)']
	0.39291053678619675
558: ['Spider-Man 2 (2004)']
	0.3704292728612526
280: ['Terminator 2: Judgment Day (1991)']
	0.3438621395970047
329: ['Jurassic Park (1993)']
	0.33784818715566267
11: ['Star Wars: Episode IV - A New Hope (1977)']
	0.32232658528473557
1891: ['Star Wars: Episode V - The Empire Strikes Back (1980)']
	0.31577146202964645
98: ['Gl

In [320]:
ids_3806 = [
    k for k, v in get_ratings(156)
    if k in [
        68,
        63,
        1891,
        603,
        99,
        43
    ]
]


print(ids_3806)


603: ['The Matrix (1999)']
	1.0968619041820322
604: ['The Matrix Reloaded (2003)']
	0.9723078744163969
550: ['Fight Club (1999)']
	0.7485962942696884
77: ['Memento (2000)']
	0.5595789697982286
752: ['V for Vendetta (2006)']
	0.5380181093411116
453: ['A Beautiful Mind (2001)']
	0.5190045658417152
14: ['American Beauty (1999)']
	0.48695683674275264
38: ['Eternal Sunshine of the Spotless Mind (2004)']
	0.47941200994272004
11: ['Star Wars: Episode IV - A New Hope (1977)']
	0.4376003692163438
1891: ['Star Wars: Episode V - The Empire Strikes Back (1980)']
	0.4333784457885115
1892: ['Star Wars: Episode VI - Return of the Jedi (1983)']
	0.43235455739042533
641: ['Requiem for a Dream (2000)']
	0.4308545633814643
155: ['The Dark Knight (2008)']
	0.41131169022089153
194: ['Amelie (2001)']
	0.3942138535665135
680: ['Pulp Fiction (1994)']
	0.3630360306104313
153: ['Lost in Translation (2003)']
	0.3564673830870303
597: ['Titanic (1997)']
	0.32264086403256864
63: ['Twelve Monkeys (a.k.a. 12 Monkeys)

In [239]:
print(latent_user_preferences.T.columns.values)

[4768  156 5323  174 4529  783 3878  768 4469 1882 4997 2067 3806 2848
 4433 3519 5338 3947 4373 2092  525 4327 3048 2665 4940]


In [240]:
print(latent_item_features.T.columns.values)

[   11    12    13    14    22    24    38    63    77    85    98   105
   107   114   120   121   122   134   141   146   153   155   161   180
   187   194   197   238   243   268   272   274   275   278   280   329
   393   414   424   453   462   550   557   558   568   581   585   597
   601   602   603   604   607   629   640   641   664   671   672   680
   745   752   786   788   807   808   809   812   854   857   862   954
   955  1422  1572  1597  1637  1891  1892  1894  1900  2024  2164  2501
  2502  3049  4327  5503  7443  8358  8467  8587  9331  9741  9802  9806
 10020 36657 36658 36955]


In [ ]:
user_ratings: pd.DataFrame = pd.DataFrame(
    columns=latent_item_features.T.columns.values,
)
user_ratings.index.name='User ID'

print(latent_user_preferences.T.columns.values)


for u in latent_user_preferences.T.columns.values:
    user_ratings.loc[u] = pd.Series(
        [
            #_rng.random()
            predict_rating(u, i)
            for i in latent_item_features.T.columns.values
        ],
    )

In [242]:
#user_ratings

,11,12,13,14,22,24,38,63,77,85,...,8467,8587,9331,9741,9802,9806,10020,36657,36658,36955
User ID,,,,,,,,,,,,,,,,,,,,,
4768,1.075866,0.490713,-0.514578,-0.370327,-0.088369,-0.171628,0.593612,0.149423,0.416629,0.129791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,0.510767,-0.125068,-0.151969,0.517930,0.189687,-0.260572,0.199863,-0.136578,0.910234,-0.637950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323,0.045008,-0.059968,-0.036395,0.247560,0.265635,-0.153931,0.271648,-0.302412,0.186516,-0.199164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,-0.075722,-0.052166,-0.353584,1.858963,-0.063346,0.000219,0.558233,-0.097778,-0.091951,-1.233248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4529,-0.211549,-0.511296,0.149259,-0.215623,0.003485,-0.564633,-0.017807,-0.236235,0.060928,-0.399136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
783,-0.128225,-0.055324,-0.017890,0.018573,0.161786,-0.105386,0.152208,-0.043136,0.027045,0.101489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3878,0.928983,0.326437,-0.065310,-1.992433,0.032120,-0.113857,0.345319,0.071474,0.562888,0.670933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
768,-0.039545,-0.047596,0.099078,0.017030,0.050608,-0.055531,0.014937,0.002273,0.012196,0.048750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4469,-0.060360,0.019270,0.010936,0.021893,0.038589,-0.054131,0.158589,-0.018854,0.008534,-0.002490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
#user_ratings[22][3806]

0.10121064592445786

In [250]:
#user_ratings[278][3878]

nan

In [249]:
#latent_user_preferences.loc[3806]

Factor1    -0.042344
Factor2    -0.040755
Factor3    -0.055720
Factor4    -0.073726
Factor5     0.018203
Factor6     0.091940
Factor7     0.123959
Factor8    -0.157403
Factor9    -0.020388
Factor10    0.082493
Factor11    0.223057
Factor12   -0.040915
Factor13   -0.130308
Factor14   -0.021816
Factor15    0.034549
Name: 3806, dtype: float64

In [259]:

def train(user_id, item_id, rating, l_user_prefs, l_item_feats, alpha=0.0001, ):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = l_user_prefs.loc[user_id]
    l_user_prefs.loc[user_id] -= alpha * err * l_item_feats.loc[item_id]
    l_item_feats.loc[item_id] -= alpha * err * user_pref_values
    return err


def sgd(iterations, l_user_prefs, l_item_feats, u_ratings):
    """ Iterate over all users and all items and train for
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in l_user_prefs.T.columns:
            for item_id in l_item_feats.T.columns:
                rating = u_ratings[item_id][user_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating, l_user_prefs, l_item_feats)
                    error.append(err)
        mse = np.square(np.array(error)).mean()
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [260]:
num_iter = 30000
hist = sgd(num_iter, latent_user_preferences, latent_item_features, user_ratings)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10000), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/30000:	MSE=0.298896


KeyboardInterrupt: 

In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [
    zip(user_ratings.loc[i], predictions[i])
    for i in range(predictions.shape[0])
]
comparison_data: pd.DataFrame = pd.DataFrame(values)
comparison_data.columns = data.columns

In [ ]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction